In [1]:
# Step 0. Load libraries and custom modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import re
from dateutil.parser import parse
from datetime import datetime
from tqdm import tqdm
# ------------  PREPROCESING -------------
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#-------------- TRANSFORMERS --------------
import transformers
from transformers.pipelines import PIPELINE_REGISTRY
from transformers import pipeline
import evaluate
from evaluate import load
from transformers import Conversation
transformers.logging.set_verbosity_error()

/home/arnaldochm/Documents/BootCamp_DataScience/Final_Project/final_project_nlp/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-07 21:05:39.703581: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-07 21:05:39.705614: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 21:05:39.747468: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 21:05:39.748737: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimize

In [2]:
df_reduced = pd.read_csv('../data/processed/df_reduced.csv')
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15738 entries, 0 to 15737
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  15738 non-null  int64 
 1   num_row     15738 non-null  int64 
 2   text        15738 non-null  object
dtypes: int64(2), object(1)
memory usage: 369.0+ KB


In [3]:
df_reduced = df_reduced.drop(['Unnamed: 0'], axis= 1)

In [4]:
df_reduced.sample(10)

,num_row,text
3627,3627,I got this book as a gift and I am in LOVE wit...
3966,3966,I was very dissapointed with every Kevin Hogan...
15609,15609,Working with the Law is a very insightful book...
10131,10131,This book examines the lives of Albert Einstei...
1194,1194,Being a baseball fan and having visited Cooper...
1837,1837,Whew... it's over.I think its obvious these re...
1792,1792,"As a Northwesterner, I was interested in readi..."
10174,10174,loved it was an action packed very well writte...
15453,15453,"That is, by reading Mr. Clarke's new novel! Ab..."
5466,5466,"Written in 1922, this biography shows a lot of..."


In [5]:
# def clean_stopwords(text: str,stop_dict: dict)->str:
#     if text is not None:
#         words = text.split()
#         words_clean = []
#         for word in words:
#             if word not in stop_dict:
#                 words_clean.append(word)
#         result = ' '.join(words_clean)
#     else:
#         result = None
#     return result

In [6]:
# 3.10 Text To Lower
df_reduced['text_clean'] = df_reduced['text'].str.lower()
# 3.12 Extract special characters and numbers
df_reduced['text_clean'] = df_reduced['text_clean'].str.replace(r'[^a-z]', ' ', regex=True)
# 3.13 Extract numbers
# df_reduced['text_clean'] = df_reduced['text_clean'].str.replace(r'[\d]+', '', regex=True)
# 3.14 #Change multiple white spaces to a single white space
df_reduced['text_clean'] = df_reduced['text_clean'].str.replace(r'\s+',' ',regex=True)

In [7]:
# 3.15 Lemmatize Text and removing Stopwords

download("wordnet")
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/arnaldochm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
download("stopwords")
stop_words = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/arnaldochm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def lemmatize_text(words, lemmatizer = lemmatizer):
    words = words.split(' ')
    tokens = [lemmatizer.lemmatize(word) for word in words]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [word for word in tokens if len(word) > 3]
    return ' '.join(tokens)

In [10]:
df_reduced["text_clean"] = df_reduced["text_clean"].apply(lambda x: lemmatize_text(x))
df_reduced["text_clean"].sample(10)

14132    else expect jonathan kellerman well continuous...
5311     purchased modern american memoir group adult w...
8392     enjoy michener ability create interesting fict...
13889    received little house collection started readi...
13096    seattle mostly native year picked book guest c...
10084    thoroughly enjoyed reading book blackwell wond...
9567     jastrow good reference piece describes languag...
7182     usually slow reader take week sometimes month ...
13778    trying level class library past year many prob...
6343     anyone interested literary translation work ap...
Name: text_clean, dtype: object

In [11]:
df_reduced = df_reduced.drop(['text'],axis=1)

In [12]:
# 3.14 See the results
df_reduced.head(10)

,num_row,text_clean
0,0,purchased book neice cruised week loved story ...
1,1,really enjoyed book several front author terri...
2,2,interesting informative spot good book read wa...
3,3,must learn past history better appreciate toda...
4,4,began book high hope cover blurb sounded inter...
5,5,gary demar succeeded writing finest short actu...
6,6,first miss julia book found quite delightful b...
7,7,enjoying read assuming writes like think long ...
8,8,book simply great start reading able page page...
9,9,hobbit written differently lord ring time much...


In [13]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/arnaldochm/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [14]:
vaderSentimentAnalyzer = SentimentIntensityAnalyzer()

In [15]:
df_reduced.iloc[67]['text_clean']

'goethe expressed fundamental truth nothing much philosophy footnote plato plotinus year later plato eliminated platonic dabbling politics quietly separated greek mysticism emerging fundamentalism christianity became somewhat bemusedly cult figure right brian hines deal subject hugely sympathatic hope future something change time plato century close timeless truth time well according plotinus father robin'

In [16]:
vaderSentimentAnalyzer.polarity_scores(df_reduced.iloc[67]['text_clean'])

{'neg': 0.0, 'neu': 0.829, 'pos': 0.171, 'compound': 0.835}

In [17]:
df_reduced['scores']=df_reduced['text_clean'].apply(lambda body: vaderSentimentAnalyzer.polarity_scores(str(body)))
df_reduced.head()

,num_row,text_clean,scores
0,0,purchased book neice cruised week loved story ...,"{'neg': 0.0, 'neu': 0.647, 'pos': 0.353, 'comp..."
1,1,really enjoyed book several front author terri...,"{'neg': 0.116, 'neu': 0.564, 'pos': 0.32, 'com..."
2,2,interesting informative spot good book read wa...,"{'neg': 0.0, 'neu': 0.504, 'pos': 0.496, 'comp..."
3,3,must learn past history better appreciate toda...,"{'neg': 0.094, 'neu': 0.554, 'pos': 0.352, 'co..."
4,4,began book high hope cover blurb sounded inter...,"{'neg': 0.094, 'neu': 0.643, 'pos': 0.263, 'co..."


In [18]:
df_reduced['compound_sentiment']=df_reduced['scores'].apply(lambda score_dict:score_dict['compound'])
df_reduced.head()

,num_row,text_clean,scores,compound_sentiment
0,0,purchased book neice cruised week loved story ...,"{'neg': 0.0, 'neu': 0.647, 'pos': 0.353, 'comp...",0.9623
1,1,really enjoyed book several front author terri...,"{'neg': 0.116, 'neu': 0.564, 'pos': 0.32, 'com...",0.9904
2,2,interesting informative spot good book read wa...,"{'neg': 0.0, 'neu': 0.504, 'pos': 0.496, 'comp...",0.7096
3,3,must learn past history better appreciate toda...,"{'neg': 0.094, 'neu': 0.554, 'pos': 0.352, 'co...",0.9595
4,4,began book high hope cover blurb sounded inter...,"{'neg': 0.094, 'neu': 0.643, 'pos': 0.263, 'co...",0.9933


In [19]:
df_reduced = df_reduced.drop(['scores'], axis=1)
df_reduced.sample(10)

,num_row,text_clean,compound_sentiment
8362,8362,story original unlike haunted house story feel...,0.4215
7980,7980,noted dance writer critic robert greskovic han...,0.6369
2272,2272,evaluating hobbit naturally compare later lord...,0.9916
10894,10894,professor hardwigg find note written arne sakn...,0.9169
8685,8685,read eragon young writer paolini wrote novel p...,0.9607
11962,11962,living inner city neighborhood heather gemmen ...,-0.9839
2808,2808,gayle wigglesworth left world began education ...,0.9822
9483,9483,book good delay respected right christmas perf...,0.9442
9394,9394,real winner word word captivating beginning fe...,0.9652
943,943,helpful descriptive beginning experienced shri...,0.4215


In [20]:
df_reduced.to_csv('../data/processed/df_reduced_with_sentiment.csv')